# Build a cuisine recommender

In [1]:
import pandas as pd

data = pd.read_csv('../data/cleaned_cuisines.csv')
X = data.iloc[:, 2:]
y = data[['cuisine']]

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
model = SVC(kernel='linear', C=10, probability=True,random_state=0)
model.fit(X_train,y_train.values.ravel())

y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.74      0.68      0.71       240
      indian       0.89      0.90      0.89       263
    japanese       0.71      0.81      0.76       212
      korean       0.88      0.75      0.81       253
        thai       0.76      0.83      0.79       231

    accuracy                           0.80      1199
   macro avg       0.79      0.79      0.79      1199
weighted avg       0.80      0.80      0.80      1199



In [3]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())